In [8]:
%pip install numpy plotly

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np

In [123]:
def create_cube(x, y, z):
    return np.array([
        [0, 0, 0],
        [x, 0, 0],
        [x, y, 0],
        [0, y, 0],
        [0, 0, z],
        [x, 0, z],
        [x, y, z],
        [0, y, z]
    ])

def translate_cube(cube, x, y, z):
    return cube + np.array([x, y, z])

def cube_faces(cube):
    return np.array([
        [cube[0], cube[1], cube[2], cube[3]],
        [cube[4], cube[5], cube[6], cube[7]],
        [cube[0], cube[1], cube[5], cube[4]],
        [cube[2], cube[3], cube[7], cube[6]],
        [cube[1], cube[2], cube[6], cube[5]],
        [cube[4], cube[7], cube[3], cube[0]]
    ])

def calculate_center_of_mass(points, weights=None):
    if weights is None:
        weights = np.ones(len(points))
    total_weight = np.sum(weights)

    x_coords = np.array([point[0] for point in points])
    y_coords = np.array([point[1] for point in points])
    z_coords = np.array([point[2] for point in points])

    center_x = np.sum(weights * x_coords) / total_weight
    center_y = np.sum(weights * y_coords) / total_weight
    center_z = np.sum(weights * z_coords) / total_weight

    return center_x, center_y, center_z

    

In [125]:
cube = create_cube(1, 2, 3)
print(cube)
print(translate_cube(cube, 1, 1, 1))
print(cube_faces(cube).T)
print(calculate_center_of_mass(cube))

[[0 0 0]
 [1 0 0]
 [1 2 0]
 [0 2 0]
 [0 0 3]
 [1 0 3]
 [1 2 3]
 [0 2 3]]
[[1 1 1]
 [2 1 1]
 [2 3 1]
 [1 3 1]
 [1 1 4]
 [2 1 4]
 [2 3 4]
 [1 3 4]]
[[[0 0 0 1 1 0]
  [1 1 1 0 1 0]
  [1 1 1 0 1 0]
  [0 0 0 1 1 0]]

 [[0 0 0 2 0 0]
  [0 0 0 2 2 2]
  [2 2 0 2 2 2]
  [2 2 0 2 0 0]]

 [[0 3 0 0 0 3]
  [0 3 0 0 0 3]
  [0 3 3 3 3 0]
  [0 3 3 3 3 0]]]
(0.5, 1.0, 1.5)


In [128]:
import plotly.graph_objects as go


def display_object(*args):
  mesh = []

  for points in args:
    # x, y, z = points.T
    # mesh.append(go.Scatter3d(
    #   x=x,
    #   y=y,
    #   z=z,
    #   mode='markers',
    # ))

    x_center, y_center, z_center = calculate_center_of_mass(points)

    mesh.append(go.Scatter3d(
      x=[x_center],
      y=[y_center],
      z=[z_center],
      mode='markers',
    ))
    

    for face in cube_faces(points):
      x, y, z = face.T
      mesh.append(go.Scatter3d(
        x=np.append(x, x[0]),
        y=np.append(y, y[0]),
        z=np.append(z, z[0]),
        mode='markers+lines',
      ))

  layout = go.Layout(
      title='',
      autosize=True,
      margin={'l': 0, 'r': 0, 'b': 0, 't': 0, "pad": 0},
      scene=dict(
          xaxis=dict(title='X-axis'),
          yaxis=dict(title='Y-axis'),
          zaxis=dict(title='Z-axis')
      )
  )
  fig = go.Figure(data=mesh, layout=layout)
  fig.show()

In [131]:
display_object(cube, translate_cube(cube, 1, 3, 1))

In [194]:
def random_cube():
  return create_cube(np.random.randint(1, 10), np.random.randint(1, 10), np.random.randint(1, 10))

def random_translate(cube):
  return translate_cube(cube, np.random.randint(0, 10), np.random.randint(0, 10), np.random.randint(0, 10))

def random_cube_list(n):
  return [random_translate(random_cube()) for _ in range(n)]

def cube_collision(cube1, cube2):
    # cube1 and cube2 should be 3D arrays with shape (8, 3) containing the coordinates of their respective vertices

    # Get the minimum and maximum coordinates of the bounding boxes for both cubes
    cube1_min_coords = np.min(cube1, axis=0)
    cube1_max_coords = np.max(cube1, axis=0)

    cube2_min_coords = np.min(cube2, axis=0)
    cube2_max_coords = np.max(cube2, axis=0)

    # Check for collision in each dimension (X, Y, Z)
    for dim in range(3):
        if cube1_max_coords[dim] <= cube2_min_coords[dim] or cube1_min_coords[dim] >= cube2_max_coords[dim]:
            # No overlap in this dimension, so the cubes are not colliding
            return False

    # If no dimension had no overlap, then the cubes are colliding
    return True

def calculate_distance_between_cubes(cube1, cube2):
    # cube1 and cube2 should be 3D arrays with shape (8, 3) containing the coordinates of their respective vertices

    min_distance = float('inf')

    # Calculate distances between all pairs of vertices
    for vertex1 in cube1:
        for vertex2 in cube2:
            distance = np.linalg.norm(vertex1 - vertex2)
            min_distance = min(min_distance, distance)

    return min_distance

In [195]:
cube1 = create_cube(1,1,1)#random_translate(random_cube())
cube2 = translate_cube(create_cube(1,2,3), 2,0,0) #random_translate(random_cube())

print(cube_collision(cube1, cube2))
print(calculate_distance_between_cubes(cube1, cube2))
display_object(cube1, cube2)

False
1.0


In [200]:
cube1 = random_cube()
cube2 = random_translate(random_cube())
<
print(cube1)

print(cube_collision(cube1, cube2))
print(calculate_distance_between_cubes(cube1, cube2))
display_object(cube1, cube2)

[[0 0 0]
 [6 0 0]
 [6 9 0]
 [0 9 0]
 [0 0 2]
 [6 0 2]
 [6 9 2]
 [0 9 2]]
False
6.48074069840786


In [147]:
cube1 = create_cube(1, 1, 1)
cube2 = create_cube(2, 2, 2)

cube_collsion(cube1, cube2)

False